In [16]:
import os
import sys
import time
import urllib.request
import datetime
import time
import json
import re
import pandas as pd
import numpy as np
import shutil
import warnings
import copy
from kiwipiepy import Kiwi
from kiwipiepy.utils import Stopwords
warnings.filterwarnings(action = 'ignore')

In [17]:
# 상위 폴더 경로 설정
base_folder_path = 'G://내 드라이브//4-1//텍데분//크롤링//크롤링폴더'  

# 하위 폴더 이름 목록
subfolders = ['국민의힘', '날씨', '민주당', '의대', '전쟁', '정부','구글','금리','네이버','부동산','삼성','애플','주식']

# 각 폴더별로 데이터프레임을 저장할 딕셔너리
dfs = {}

for subfolder in subfolders:
    # 각 하위 폴더 경로 설정
    folder_path = os.path.join(base_folder_path, subfolder)
    
    # 폴더 내 모든 파일 목록을 가져옴
    file_list = os.listdir(folder_path)
    
    # CSV 파일만 필터링
    csv_files = [file for file in file_list if file.endswith('.csv')]
    
    # 모든 CSV 파일을 하나의 데이터프레임으로 합침
    df_list = []

    for csv_file in csv_files:
        file_path = os.path.join(folder_path, csv_file)
        df = pd.read_csv(file_path)
        df_list.append(df)

    # 하나의 데이터프레임으로 병합
    combined_df = pd.concat(df_list, ignore_index=True)
    
    # 결과 데이터프레임을 딕셔너리에 저장
    dfs[subfolder] = combined_df

# 필요시 각 데이터프레임을 별도의 CSV 파일로 저장
for subfolder, df in dfs.items():
    output_path = os.path.join(base_folder_path, f'{subfolder}_combined.csv')
    df.to_csv(output_path, index=False)


In [18]:
df_국민의힘 = pd.read_csv('G://내 드라이브//4-1//텍데분//크롤링//크롤링폴더//국민의힘_combined.csv')
df_날씨 = pd.read_csv('G://내 드라이브//4-1//텍데분//크롤링//크롤링폴더//날씨_combined.csv')
df_민주당 = pd.read_csv('G://내 드라이브//4-1//텍데분//크롤링//크롤링폴더//민주당_combined.csv')
df_의대 = pd.read_csv('G://내 드라이브//4-1//텍데분//크롤링//크롤링폴더//의대_combined.csv')
df_전쟁 = pd.read_csv('G://내 드라이브//4-1//텍데분//크롤링//크롤링폴더//전쟁_combined.csv')
df_정부 = pd.read_csv('G://내 드라이브//4-1//텍데분//크롤링//크롤링폴더//정부_combined.csv')
df_주식 = pd.read_csv('G://내 드라이브//4-1//텍데분//크롤링//크롤링폴더//주식_combined.csv')
df_애플 = pd.read_csv('G://내 드라이브//4-1//텍데분//크롤링//크롤링폴더//애플_combined.csv')
df_삼성 = pd.read_csv('G://내 드라이브//4-1//텍데분//크롤링//크롤링폴더//삼성_combined.csv')
df_부동산 = pd.read_csv('G://내 드라이브//4-1//텍데분//크롤링//크롤링폴더//부동산_combined.csv')
df_네이버 = pd.read_csv('G://내 드라이브//4-1//텍데분//크롤링//크롤링폴더//네이버_combined.csv')
df_금리 = pd.read_csv('G://내 드라이브//4-1//텍데분//크롤링//크롤링폴더//금리_combined.csv')
df_구글 = pd.read_csv('G://내 드라이브//4-1//텍데분//크롤링//크롤링폴더//구글_combined.csv')

In [13]:
stopword_ko = [
    '가', '가까스로', '가령', '각', '각각', '각자', '각종', '갖고말하자면', '같다', '같이', '개의치않고', '거니와', '거바', '거의', '것', '것과 같이', '것들',
    '게다가', '게우다', '겨우', '견지에서', '결과에 이르다', '결국', '결론을 낼 수 있다', '겸사겸사', '고려하면', '고로', '곧', '공동으로', '과', '과연', '관계가 있다',
    '관계없이', '관련이 있다', '관하여', '관한', '관해서는', '구', '구체적으로', '구토하다', '그', '그들', '그때', '그래', '그래도', '그래서', '그러나', '그러니', '그러니까',
    '그러면', '그러므로', '그러한즉', '그런 까닭에', '그런데', '그런즉', '그럼', '그럼에도 불구하고', '그렇게 함으로써', '그렇지', '그렇지 않다면', '그렇지 않으면', '그렇지만', '그렇지않으면',
    '그리고', '그리하여', '그만이다', '그에 따르는', '그위에', '그저', '그중에서', '그치지 않다', '근거로', '근거하여', '기대여', '기점으로', '기준으로', '기타', '까닭으로', '까악',
    '까지', '까지 미치다', '까지도', '꽈당', '끙끙', '끼익', '나', '나머지는', '남들', '남짓', '너', '너희', '너희들', '네', '넷', '년', '논하지 않다', '놀라다', '누가 알겠는가',
    '누구', '다른', '다른 방면으로', '다만', '다섯', '다소', '다수', '다시 말하자면', '다시말하면', '다음', '다음에', '다음으로', '단지', '답다', '당신', '당장', '대로 하다',
    '대하면', '대하여', '대해 말하자면', '대해서', '댕그', '더구나', '더군다나', '더라도', '더불어', '더욱더', '더욱이는', '도달하다', '도착하다', '동시에', '동안', '된바에야',
    '된이상', '두번째로', '둘', '둥둥', '뒤따라', '뒤이어', '든간에', '들', '등', '등등', '딩동', '따라', '따라서', '따위', '따지지 않다', '딱', '때', '때가 되어', '때문에', '또',
    '또한', '뚝뚝', '라 해도', '령', '로', '로 인하여', '로부터', '로써', '륙', '를', '마음대로', '마저', '마저도', '마치', '막론하고', '만 못하다', '만약', '만약에', '만은 아니다',
    '만이 아니다', '만일', '만큼', '말하자면', '말할것도 없고', '매', '매번', '메쓰겁다', '몇', '모', '모두', '무렵', '무릎쓰고', '무슨', '무엇', '무엇때문에', '물론', '및', '바꾸어말하면',
    '바꾸어말하자면', '바꾸어서 말하면', '바꾸어서 한다면', '바꿔 말하면', '바로', '바와같이', '밖에 안된다', '반대로', '반대로 말하자면', '반드시', '버금', '보는데서', '보다더', '보드득',
    '본대로', '봐', '봐라', '부류의 사람들', '부터', '불구하고', '불문하고', '붕붕', '비걱거리다', '비교적', '비길수 없다', '비로소', '비록', '비슷하다', '비추어 보아', '비하면', '뿐만 아니라',
    '뿐만아니라', '뿐이다', '삐걱', '삐걱거리다', '사', '삼', '상대적으로 말하자면', '생각한대로', '설령', '설마', '설사', '셋', '소생', '소인', '솨', '쉿', '습니까', '습니다', '시각', '시간',
    '시작하여', '시초에', '시키다', '실로', '심지어', '아', '아니', '아니나다를가', '아니라면', '아니면', '아니었다면', '아래윗', '아무거나', '아무도', '아야', '아울러', '아이', '아이고',
    '아이고', '아이야', '아이쿠', '아하', '아홉', '안 그러면', '않기 위하여', '않기 위해서', '알 수 있다', '알았어', '앗', '앞에서', '앞의것', '야', '약간', '양자', '어', '어기여차', '어느',
    '어느 년도', '어느것', '어느곳', '어느때', '어느쪽', '어느해', '어디', '어때', '어떠한', '어떤', '어떤것', '어떤것들', '어떻게', '어떻해', '어이', '어째서', '어쨋든', '어쩔수 없다',
    '어찌', '어찌됏든', '어찌됏어', '어찌하든지', '어찌하여', '언제', '언젠가', '얼마', '얼마 안 되는 것', '얼마간', '얼마나', '얼마든지', '얼마만큼', '얼마큼', '엉엉', '에', '에 가서', '에 달려 있다',
    '에 대해', '에 있다', '에 한하다', '에게', '에서', '여', '여기', '여덟', '여러분', '여보시오', '여부', '여섯', '여전히', '여차', '연관되다', '연이서', '영', '영차', '옆사람', '예', '예를 들면',
    '예를 들자면', '예컨대', '예하면', '오', '오로지', '오르다', '오자마자', '오직', '오호', '오히려', '와', '와 같은 사람들', '와르르', '와아', '왜', '왜냐하면', '외에도', '요만큼', '요만한 것',
    '요만한걸', '요컨대', '우르르', '우리', '우리들', '우선', '우에 종합한것과같이', '운운', '월', '위에서 서술한바와같이', '위하여', '위해서', '윙윙', '육', '으로', '으로 인하여', '으로서', '으로써',
    '을', '응', '응당', '의', '의거하여', '의지하여', '의해', '의해되다', '의해서', '이', '이 되다', '이 때문에', '이 밖에', '이 외에', '이 정도의', '이것', '이곳', '이때', '이라면', '이래', '이러이러하다',
    '이러한', '이런', '이럴정도로', '이렇게 많은 것', '이렇게되면', '이렇게말하자면', '이렇구나', '이로 인하여', '이르기까지', '이리하여', '이만큼', '이번', '이봐', '이상', '이어서', '이었다', '이와 같다',
    '이와 같은', '이와 반대로', '이와같다면', '이외에도', '이용하여', '이유만으로', '이젠', '이지만', '이쪽', '이천구', '이천육', '이천칠', '이천팔', '인 듯하다', '인젠', '일', '일것이다', '일곱', '일단', '일때',
    '일반적으로', '일지라도', '임에 틀림없다', '입각하여', '입장에서', '잇따라', '있다', '자', '자기', '자기집', '자마자', '자신', '잠깐', '잠시', '저', '저것', '저것만큼', '저기', '저쪽', '저희', '전부', '전자',
    '전후', '점에서 보아', '정도에 이르다', '제', '제각기', '제외하고', '조금', '조차', '조차도', '졸졸', '좀', '좋아', '좍좍', '주룩주룩', '주저하지 않고', '줄은 몰랏다', '줄은모른다', '중에서', '중의하나', '즈음하여',
    '즉', '즉시', '지든지', '지만', '지말고', '진짜로', '쪽으로', '차라리', '참', '참나', '첫번째로', '쳇', '총적으로', '총적으로 말하면', '총적으로 보면', '칠', '콸콸', '쾅쾅', '쿵', '타다', '타인', '탕탕', '토하다', '통하여',
    '툭', '퉤', '틈타', '팍', '팔', '퍽', '펄렁', '하', '하게될것이다', '하게하다', '하겠는가', '하고 있다', '하고있었다', '하곤하였다', '하구나', '하기 때문에', '하기 위하여', '하기는한데', '하기만 하면', '하기보다는',
    '하기에', '하나', '하느니', '하는 김에', '하는 편이 낫다', '하는것도', '하는것만 못하다', '하는것이 낫다', '하는바', '하더라도', '하도다', '하도록시키다', '하도록하다', '하든지', '하려고하다', '하마터면', '하면 할수록',
    '하면된다', '하면서', '하물며', '하여금', '하여야', '하자마자', '하지 않는다면', '하지 않도록', '하지마', '하지마라', '하지만', '하하', '한 까닭에', '한 이유는', '한 후', '한다면', '한다면 몰라도', '한데', '한마디', '한적이있다',
    '한켠으로는', '한항목', '할 따름이다', '할 생각이다', '할 줄 안다', '할 지경이다', '할 힘이 있다', '할때', '할만하다', '할망정', '할뿐', '할수있다', '할수있어', '할줄알다', '할지라도', '할지언정', '함께', '해도된다', '해도좋다',
    '해봐요', '해서는 안된다', '해야한다', '해요', '했어요', '향하다', '향하여', '향해서', '허', '허걱', '허허', '헉', '헉헉', '헐떡헐떡', '형식으로 쓰여', '혹시', '혹은', '혼자', '훨씬', '휘익', '휴', '흐흐', '흥', '힘입어',
]

# 전처리

In [14]:
def preprocess_text(df, column_name):

    # 전처리 전 샘플 개수 출력
    print(' 전처리 전 학습용 샘플의 개수 :', len(df))

    # 중복 제거
    df.drop_duplicates(subset=[column_name], inplace=True)

    # 공백은 empty 값으로 변경
    df[column_name] = df[column_name].str.replace('^ +', "", regex=True)

    # 줄바꿈 문자, 탭 문자, 역슬래시 문자 제거
    df[column_name] = df[column_name].str.replace('\n', '', regex=False)
    df[column_name] = df[column_name].str.replace('\t', '', regex=False)
    df[column_name] = df[column_name].str.replace('\\', '', regex=False)

    # 특수 문자 제거
    df[column_name] = df[column_name].str.replace('[^\w\s]', '', regex=True)

    # 모든 텍스트를 소문자로 변환
    df[column_name] = df[column_name].str.lower()

    # 공백 문자열을 NaN으로 변경
    df[column_name].replace('', np.nan, inplace=True)

    # NaN 값 제거
    df = df.dropna(how='any')

    # 인덱스 리셋
    df = df.reset_index(drop=True)

    # 전처리 후 샘플 개수 출력
    print( '전처리 후 학습용 샘플의 개수 :', len(df))
    
    return df


# df_list =['df_국민의힘','df_날씨','df_민주당','df_의대','df_전쟁','df_정부']

df_국민의힘 = preprocess_text(df_국민의힘 , 'body')
df_날씨 = preprocess_text(df_날씨 , 'body')
df_민주당 = preprocess_text(df_민주당 , 'body')
df_의대 = preprocess_text(df_의대 , 'body')
df_전쟁 = preprocess_text(df_전쟁 , 'body')
df_정부 = preprocess_text(df_정부 , 'body')
df_주식 = preprocess_text(df_주식 , 'body')
df_애플 = preprocess_text(df_애플 , 'body')
df_삼성 = preprocess_text(df_삼성 , 'body')
df_부동산 = preprocess_text(df_부동산 , 'body')
df_네이버 = preprocess_text(df_네이버 , 'body')
df_금리 = preprocess_text(df_금리 , 'body')
df_구글 = preprocess_text(df_구글 , 'body')

df_국민의힘 = preprocess_text(df_국민의힘 , 'title')
df_날씨 = preprocess_text(df_날씨 , 'title')
df_민주당 = preprocess_text(df_민주당 , 'title')
df_의대 = preprocess_text(df_의대 , 'title')
df_전쟁 = preprocess_text(df_전쟁 , 'title')
df_정부 = preprocess_text(df_정부 , 'title')
df_주식 = preprocess_text(df_주식 , 'title')
df_애플 = preprocess_text(df_애플 , 'title')
df_삼성 = preprocess_text(df_삼성 , 'title')
df_부동산 = preprocess_text(df_부동산 , 'title')
df_네이버 = preprocess_text(df_네이버 , 'title')
df_금리 = preprocess_text(df_금리 , 'title')
df_구글 = preprocess_text(df_구글 , 'title')

 전처리 전 학습용 샘플의 개수 : 4776
전처리 후 학습용 샘플의 개수 : 1775
 전처리 전 학습용 샘플의 개수 : 4528
전처리 후 학습용 샘플의 개수 : 2252
 전처리 전 학습용 샘플의 개수 : 4886
전처리 후 학습용 샘플의 개수 : 1967
 전처리 전 학습용 샘플의 개수 : 5289
전처리 후 학습용 샘플의 개수 : 2571
 전처리 전 학습용 샘플의 개수 : 4456
전처리 후 학습용 샘플의 개수 : 1592
 전처리 전 학습용 샘플의 개수 : 5290
전처리 후 학습용 샘플의 개수 : 3146
 전처리 전 학습용 샘플의 개수 : 3849
전처리 후 학습용 샘플의 개수 : 1296
 전처리 전 학습용 샘플의 개수 : 3785
전처리 후 학습용 샘플의 개수 : 702
 전처리 전 학습용 샘플의 개수 : 3643
전처리 후 학습용 샘플의 개수 : 2054
 전처리 전 학습용 샘플의 개수 : 3634
전처리 후 학습용 샘플의 개수 : 1442
 전처리 전 학습용 샘플의 개수 : 4036
전처리 후 학습용 샘플의 개수 : 952
 전처리 전 학습용 샘플의 개수 : 3958
전처리 후 학습용 샘플의 개수 : 1561
 전처리 전 학습용 샘플의 개수 : 3735
전처리 후 학습용 샘플의 개수 : 748
 전처리 전 학습용 샘플의 개수 : 1775
전처리 후 학습용 샘플의 개수 : 1645
 전처리 전 학습용 샘플의 개수 : 2252
전처리 후 학습용 샘플의 개수 : 1965
 전처리 전 학습용 샘플의 개수 : 1967
전처리 후 학습용 샘플의 개수 : 1828
 전처리 전 학습용 샘플의 개수 : 2571
전처리 후 학습용 샘플의 개수 : 2353
 전처리 전 학습용 샘플의 개수 : 1592
전처리 후 학습용 샘플의 개수 : 1466
 전처리 전 학습용 샘플의 개수 : 3146
전처리 후 학습용 샘플의 개수 : 2969
 전처리 전 학습용 샘플의 개수 : 1296
전처리 후 학습용 샘플의 개수 : 1222
 전처리 전 학습용 샘플의 개수 : 702

In [15]:
kiwi = Kiwi(typos='basic')

kiwi.add_user_word('국민의힘','NNP')
kiwi.add_user_word('민주당','NNP')
kiwi.add_user_word('의대','NNP')
kiwi.add_user_word('정부','NNP')
kiwi.add_user_word('전쟁','NNP')
kiwi.add_user_word('날씨','NNP')


# Stopwords 객체 초기화
custom_stopwords = Stopwords()

# 불용어 추가
for word in stopword_ko:
    custom_stopwords.add(word)

def preprocess_korean(text, analyzer=kiwi, stopwords=custom_stopwords):
    my_text = copy.copy(text)
    my_text = my_text.replace('\n', ' ') # (1) 줄바꿈 문자 제거
    my_text = kiwi.space(my_text) # (2) 띄어쓰기 교정
    sents = kiwi.split_into_sents(my_text) # (3) 문장 토큰화
    p = re.compile('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]')
    all_result = []
    for sent in sents:
        token_result = kiwi.tokenize(sent.text, stopwords=stopwords) # (4) 형태소 분석 + 간단한 오타 교정
        filtered_tokens = [token.form for token in token_result if token.tag not in ['JKS', 'JKC', 'JKG', 'JKO', 'JKB', 'JKV', 'JKQ', 'JX', 'JC', 'EP', 'EF', 'EC', 'ETN', 'ETM']]  # (5) 조사 및 어미 필터링
        token_result = ' '.join(filtered_tokens)
        token_result = p.sub(' ', token_result) # (5) 특수 문자 제거 (=한글을 제외한 문자 제거)
        all_result.append(token_result) # (6) 형태소 분석한 결과를 다시 join
    
    all_result = ' '.join(all_result) # (7) 모든 문장을 하나의 string으로 join

    return all_result

In [32]:
%%time
df_국민의힘['preprocessed_body'] = df_국민의힘['body'].apply(lambda x: preprocess_korean(x))
df_날씨['preprocessed_body'] = df_날씨['body'].apply(lambda x: preprocess_korean(x))
df_민주당['preprocessed_body'] = df_민주당['body'].apply(lambda x: preprocess_korean(x))
df_의대['preprocessed_body'] = df_의대['body'].apply(lambda x: preprocess_korean(x))
df_전쟁['preprocessed_body'] = df_전쟁['body'].apply(lambda x: preprocess_korean(x))
df_정부['preprocessed_body'] = df_정부['body'].apply(lambda x: preprocess_korean(x))
df_주식['preprocessed_body'] = df_주식['body'].apply(lambda x: preprocess_korean(x))
df_애플['preprocessed_body'] = df_애플['body'].apply(lambda x: preprocess_korean(x))
df_삼성['preprocessed_body'] = df_삼성['body'].apply(lambda x: preprocess_korean(x))
df_부동산['preprocessed_body'] = df_부동산['body'].apply(lambda x: preprocess_korean(x))
df_네이버['preprocessed_body'] = df_네이버['body'].apply(lambda x: preprocess_korean(x))
df_금리['preprocessed_body'] = df_금리['body'].apply(lambda x: preprocess_korean(x))
df_구글['preprocessed_body'] = df_구글['body'].apply(lambda x: preprocess_korean(x))


df_국민의힘['preprocessed_title'] = df_국민의힘['title'].apply(lambda x: preprocess_korean(x))
df_날씨['preprocessed_title'] = df_날씨['title'].apply(lambda x: preprocess_korean(x))
df_민주당['preprocessed_title'] = df_민주당['title'].apply(lambda x: preprocess_korean(x))
df_의대['preprocessed_title'] = df_의대['title'].apply(lambda x: preprocess_korean(x))
df_전쟁['preprocessed_title'] = df_전쟁['title'].apply(lambda x: preprocess_korean(x))
df_정부['preprocessed_title'] = df_정부['title'].apply(lambda x: preprocess_korean(x))
df_주식['preprocessed_title'] = df_주식['title'].apply(lambda x: preprocess_korean(x))
df_애플['preprocessed_title'] = df_애플['title'].apply(lambda x: preprocess_korean(x))
df_삼성['preprocessed_title'] = df_삼성['title'].apply(lambda x: preprocess_korean(x))
df_부동산['preprocessed_title'] = df_부동산['title'].apply(lambda x: preprocess_korean(x))
df_네이버['preprocessed_title'] = df_네이버['title'].apply(lambda x: preprocess_korean(x))
df_금리['preprocessed_title'] = df_금리['title'].apply(lambda x: preprocess_korean(x))
df_구글['preprocessed_title'] = df_구글['title'].apply(lambda x: preprocess_korean(x))

CPU times: total: 21min 31s
Wall time: 22min


In [33]:
df_주식.to_csv("df국민의힘.csv",index = False,encoding="utf-8-sig")
df_날씨.to_csv("df날씨.csv",index = False,encoding="utf-8-sig")
df_민주당.to_csv("df민주당.csv",index = False,encoding="utf-8-sig")
df_의대.to_csv("df의대.csv",index = False,encoding="utf-8-sig")
df_전쟁.to_csv("df전쟁.csv",index = False,encoding="utf-8-sig")
df_정부.to_csv("df정부.csv",index = False,encoding="utf-8-sig")
df_주식.to_csv("df주식.csv",index = False,encoding="utf-8-sig")
df_애플.to_csv("df애플.csv",index = False,encoding="utf-8-sig")
df_삼성.to_csv("df삼성.csv",index = False,encoding="utf-8-sig")
df_부동산.to_csv("df부동산.csv",index = False,encoding="utf-8-sig")
df_네이버.to_csv("df네이버.csv",index = False,encoding="utf-8-sig")
df_금리.to_csv("df금리.csv",index = False,encoding="utf-8-sig")
df_구글.to_csv("df구글.csv",index = False,encoding="utf-8-sig")